# CDC-WONDER Scraper
## North America Land Data Assimilation System (NLDAS) Daily Air Temperatures and Heat Index (1979-2011)

In [ ]:
# If necessary, install libraries

#! pip install selenium
#! pip install webdriver_manager

In [5]:
# Load libraries

import os
import io
import time
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# CDC-WONDER URL
TARGET_PAGE = "NASA-NLDAS" # North America Land Data Assimilation System (NLDAS) Daily Air Temperatures and Heat Index (1979-2011)

#### Define Functions

In [3]:
# Set up Chrome Driver using Selenium
def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs", {"download.default_directory": os.getcwd()})  # Set download dir
    options.add_argument("--disable-gpu")  # Avoid potential GPU issues
    options.add_argument("--headless=new")  # Run in headless mode (new headless mode in Chrome 109+)
    options.add_argument("--window-size=1920,1080")  # Set window size for headless mode
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Query WONDER GUI
def submit_query(driver, state, year, month):
    try:
        print(f"Submitting query for State: {state}, Year: {year}, Month: {month}")
        driver.get(f"https://wonder.cdc.gov/{TARGET_PAGE}.html")
        timeout = 5   
        time.sleep(5)  # Ensure page loads
    
        # Selecting 'Month Day, Year' and 'County' in dropdowns
        Select(driver.find_element(By.NAME, "B_1")).select_by_value("D60.V7-level3")  # "Month Day, Year"
        Select(driver.find_element(By.NAME, "B_2")).select_by_value("D60.V2-level2")  # "County"
        
        # Select all measures
        checkboxes = driver.find_elements(By.XPATH, "//input[@type='checkbox' and starts-with(@name, 'M_')]")
        for checkbox in checkboxes:
            if not checkbox.is_selected():
                driver.execute_script("arguments[0].click();", checkbox)
        
        # Step 2: Select Location (Ensure 'States' is checked first)
        states_radio = driver.find_element(By.ID, "RO_locationD60.V2")
        if not states_radio.is_selected():
            states_radio.click()
        
        state_select = Select(driver.find_element(By.NAME, "F_D60.V2"))
        state_select.deselect_all()
        
        state_select.select_by_value(state.split(" ")[0])  # Extracting state code from "01 (Alabama)"
        
        # Step 3: Select Year and Month, ensuring 'All Days' is selected
        date_radio = driver.find_element(By.ID, "RO_datesD60.V3")
        if not date_radio.is_selected():
            date_radio.click()
        
        year_select = Select(driver.find_element(By.NAME, "V_D60.V3"))
        year_select.deselect_all()
        
        month_select = Select(driver.find_element(By.NAME, "V_D60.V4"))
        month_select.deselect_all()
        
        year_select.select_by_value(str(year))
        month_select.select_by_value(str(month))
        
        day_select = Select(driver.find_element(By.NAME, "V_D60.V8"))
        day_select.select_by_value("*All*")
        
        # Step 4: Ensure 'Ranges' is selected and leave temperature fields blank
        temp_radio = driver.find_element(By.ID, "RO_tempFF2_range")
        if not temp_radio.is_selected():
            temp_radio.click()
        
        temp_fields = ["R1_D60.V10", "R2_D60.V10", "R1_D60.V12", "R2_D60.V12", "R1_D60.V14", "R2_D60.V14"]
        for field in temp_fields:
            temp_input = driver.find_element(By.NAME, field)
            temp_input.clear()  # Ensure fields are empty
        
        # Step 5: Export Results, Set Precision, and Disable Totals/Zero Values
        export_checkbox = driver.find_element(By.ID, "export-option")
        driver.execute_script("arguments[0].scrollIntoView();", export_checkbox)
        driver.execute_script("arguments[0].click();", export_checkbox)
        
        precision_select = Select(driver.find_element(By.NAME, "O_precision"))
        precision_select.select_by_value("9")
        
        totals_checkbox = driver.find_element(By.ID, "CO_show_totals")
        if totals_checkbox.is_selected():
            totals_checkbox.click()
        zero_values_checkbox = driver.find_element(By.ID, "CO_show_zeros")
        if zero_values_checkbox.is_selected():
            zero_values_checkbox.click()
        
        # Submit Query
        send_button = driver.find_element(By.NAME, "action-Send")
        driver.execute_script("arguments[0].click();", send_button)  # Click using JS to ensure execution
        
        time.sleep(10)  # Allow extra time for download to complete
    except Exception as e:
        print(f"Error during query submission: {e}. Restarting WebDriver and retrying...")
        driver.quit()  # Close existing session
        driver = setup_driver()  # Restart WebDriver
        submit_query(driver, state, year, month)  # Retry only this query
        
# Clean downloaded text file
def txt_cleaner():
    # Get the list of all .txt files in the current working directory
    txt_files = [f for f in os.listdir() if f.endswith('.txt')]
    
    for txt_file in txt_files:
        with open(txt_file, 'r') as file:
            lines = file.readlines()
        
        # Find the index where the footer starts (look for the line '"---"')
        footer_start = None
        for i, line in enumerate(lines):
            if line.strip() == '"---"':
                footer_start = i
                break
        
        # If a footer is found, remove it
        if footer_start is not None:
            with open(txt_file, 'w') as file:
                file.writelines(lines[:footer_start])
        
# Append download to CSV       
def append_csv(csv_filename):
    # Get the list of all .txt files in the current working directory
    txt_files = [f for f in os.listdir() if f.endswith('.txt')]
    
    # Read the header from the CSV file (if it exists)
    header = None
    if os.path.exists(csv_filename):
        with open(csv_filename, 'r') as file:
            reader = csv.reader(file)
            header = next(reader, None)  # Read the header row
    
    # Open the CSV file in append mode
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # If the CSV file is empty or doesn't exist, write the header
        if header is None:
            # Assume the first .txt file has the same header structure
            with open(txt_files[0], 'r') as first_txt:
                header = first_txt.readline().strip().split('\t')
            writer.writerow(header)
        
        # Loop through each .txt file and append its rows to the CSV
        for txt_file in txt_files:
            with open(txt_file, 'r') as file:
                # Skip the header row in the .txt file
                next(file)
                for line in file:
                    # Split the tab-separated line into a list
                    row = line.strip().split('\t')
                    # Strip quotation marks from each cell in the row
                    row = [cell.replace('"', '') for cell in row]
                    # Write the cleaned row to the CSV
                    writer.writerow(row)
    
    # Delete all .txt files after appending their data to the CSV
    for txt_file in txt_files:
        os.remove(txt_file)
        #print(f"Deleted {txt_file}.")
    
    print(f"Data from {len(txt_files)} .txt files appended to {csv_filename}.")

#### Main Workflow

In [ ]:
def main():
    driver = setup_driver()
    states = [
        "01 (Alabama)", 
        "04 (Arizona)", 
        "05 (Arkansas)", 
        "06 (California)", 
        "08 (Colorado)",
        "09 (Connecticut)", 
        "10 (Delaware)", 
        "11 (District of Columbia)", 
        "12 (Florida)", 
        "13 (Georgia)",
        "16 (Idaho)", 
        "17 (Illinois)", 
        "18 (Indiana)", 
        "19 (Iowa)", 
        "20 (Kansas)", 
        "21 (Kentucky)",
        "22 (Louisiana)", 
        "23 (Maine)", 
        "24 (Maryland)", 
        "25 (Massachusetts)", 
        "26 (Michigan)",
        "27 (Minnesota)", 
        "28 (Mississippi)", 
        "29 (Missouri)", 
        "30 (Montana)", 
        "31 (Nebraska)",
        "32 (Nevada)", 
        "33 (New Hampshire)", 
        "34 (New Jersey)", 
        "35 (New Mexico)", 
        "36 (New York)",
        "37 (North Carolina)", 
        "38 (North Dakota)", 
        "39 (Ohio)", 
        "40 (Oklahoma)", 
        "41 (Oregon)",
        "42 (Pennsylvania)", 
        "44 (Rhode Island)", 
        "45 (South Carolina)", 
        "46 (South Dakota)",
        "47 (Tennessee)", 
        "48 (Texas)", 
        "49 (Utah)", 
        "50 (Vermont)", 
        "51 (Virginia)", 
        "53 (Washington)",
        "54 (West Virginia)", 
        "55 (Wisconsin)", 
        "56 (Wyoming)"
    ]
    years = list(range(1979, 2012))
    months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]  
    
    for state in states:
        # Extract the two-digit state code from the state string
        state_code = state.split(" ")[0]  # Splits "04 (Arizona)" and takes "04"
        csv_filename = f"{state_code}_NLDAS_dailyTemp_1979-2012.csv"
        
        # Create CSV with correct fields (only if it doesn't already exist)
        if not os.path.exists(csv_filename):
            headers = [
                "Month Day, Year", "Month Day, Year Code", "County", "County Code", 
                "Avg Daily Max Air Temperature (F)", "Record Count for Daily Max Air Temp (F)", 
                "Min Temp for Daily Max Air Temp (F)", "Max Temp for Daily Max Air Temp (F)", 
                "Avg Daily Min Air Temperature (F)", "Record Count for Daily Min Air Temp (F)", 
                "Min Temp for Daily Min Air Temp (F)", "Max Temp for Daily Min Air Temp (F)", 
                "Avg Daily Max Heat Index (F)", "Record Count for Daily Max Heat Index (F)", 
                "Min for Daily Max Heat Index (F)", "Max for Daily Max Heat Index (F)", 
                "Daily Max Heat Index (F) % Coverage"
            ]
            pd.DataFrame(columns=headers).to_csv(csv_filename, index=False)
        
        for year in years:
            for month in months:
                submit_query(driver, state, year, month)
                txt_cleaner()
                append_csv(csv_filename)  # Pass the CSV filename to append_csv()
                
    driver.quit()
    print("Data collection complete.")

if __name__ == "__main__":
    main()

Submitting query for State: 01 (Alabama), Year: 1979, Month: 1
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 2
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 3
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 4
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 5
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 6
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 7
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1979, Month: 8
Data from 1 .

Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 4
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 5
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 6
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 7
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 8
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 9
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1984, Month: 10
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submit

Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 6
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 7
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 8
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 9
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 10
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 11
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Submitting query for State: 01 (Alabama), Year: 1989, Month: 12
Data from 1 .txt files appended to 01_NLDAS_dailyTemp_1979-2012.csv.
Subm